In [1]:
from common import *

In [ ]:
import pandas as pd
N_TRAIN, N_TEST = 623817, 640430

In [3]:
# generate hdf5 from original files
if __name__ == '__main__':
    from decimal import Decimal
    df = pd.read_csv('train.csv', index_col=0, dtype={'y': str})
    df.index = df.index - N_TRAIN
    df.y = (df.y.map(Decimal) * 10000).astype(float)
    df.to_hdf(top_dir + '/data/given/train.hdf5', mode='w', key='df')
    dg = pd.read_csv('test.csv', index_col=0)
    dg.to_hdf(top_dir + '/data/given/test.hdf5', mode='w', key='dg')

In [ ]:
try:
    df = pd.read_hdf(top_dir + 'data/given/train.hdf5')
    dg = pd.read_hdf(top_dir + 'data/given/test.hdf5')
except FileNotFoundError:
    if __name__ != '__main__':
        raise
    df = pd.read_csv(top_dir + 'data/given/train.csv', index_col=0)
    df.index = df.index - N_TRAIN
    df.y *= 10000
    dg = pd.read_csv(top_dir + 'data/given/test.csv', index_col=0)
dh = pd.concat([df, dg])
dh['x3'] = dh['x3A x3B x3C x3D x3E'.split()].mean(axis=1)
dh['x3l'] = dh['x3A x3B x3C x3D x3E'.split()].pipe(np.log).mean(axis=1).pipe(np.exp)
dh['wy'] = dh.Weight * dh.y
dh['Index'] = dh.index
dh['Count'] = 1
df, dg = dh.loc[:-1], dh.loc[0:]

In [ ]:
def make_stock_wide(long):
    orig_index, orig_type = long.index, type(long)
    ref = dh.loc[long.index]
    try:
        long.index = pd.MultiIndex.from_arrays([ref.Day, ref.Stock])
        wide = long.unstack() # unstacked level is autosorted
        # # changed my mind, no swapping levels for columns to save time/space
        #if issubclass(orig_type, pd.DataFrame):
        #    wide = wide.swaplevel(axis=1)
    finally:
        long.index = orig_index
    return wide
makesw = make_stock_wide

In [13]:
def index_n_valid_days_filter(dh, n_valid):
    return dh.groupby('Stock').y.transform(lambda x: x.count() >= n_valid).rename('Valid')
index_nvdf = index_n_valid_days_filter

def stock_n_valid_days_filter(dh, n_valid):
    return dh.groupby('Stock').y.agg(lambda x: x.count() >= n_valid).rename('Valid')
stock_nvdf = stock_n_valid_days_filter

def day_filter(df, n_valid, stock='Stock'):
    '''Params - df: `DataFrame` with a 'Stock' and 'y' column,
                n_valid: minimum number of valid entries to keep a stock,
                stock: just 'Stock', or the `Series` containing the groupby value
    Filters `df` for only stock entries that mean minimum `n_valid` observations in the `y` variable'''
    return df.groupby('Stock').filter(lambda x: x.y.count() >= n_valid)

dff = day_filter(df, 246)
dhh = day_filter(dh, 1)
dhf = day_filter(dh, 246)

In [64]:
if __name__ == '__main__':
    print(dh.index[0])
    print(set(df.Stock.unique()) ^ set(range(3023)))
    print(set(dh.index) - set(dhh.index))
dfh = dh.loc[list(range(dh.index[0], 0)) + [43289]]
dfh.index = list(dfh.index[:-1]) + [-623818]
dfh.sort_index(inplace=True)
dfh.iloc[0, 0] = -1
dfh.iloc[0, 1] = 1
dfh.iloc[0, 2] = 1917
for j in range(4, 15):
    dfh.iloc[0, j] = np.nan
dfh.iloc[0, 17] = -623818
dfh.iloc[0, 18] = 0

-623817
{1917}
{43289}


In [5]:
N_TRAIN, N_TEST = len(df), len(dg) # should be 623817, 640430
N_STOCK = dh.Stock.nunique() # should be 3023

In [6]:
xfeats = 'x0 x1 x2 x3A x3B x3C x3D x3E x4 x5 x6'.split()
x_feats = 'x0 x1 x2 x3 x4 x5 x6'.split()
x3feats = 'x3A x3B x3C x3D x3E'.split()
idfeats = 'Market Stock'.split()

### Matrix stuff

In [ ]:
def def_WY(namespace, dh):
    '''Params - namespace: e.g. `globals()`, dh: `dh` or sub-DataFrame of it (`dh` itself is recommended)
    Put convenience matrix variables into `namespace`'''
    class var_map(O()):
        W = dh.Weight.pipe(makesw)
        Wnrm = W.pipe(lambda w: w / w.sum())
        Wa = W.fillna(0)
        Wnrma = Wnrm.fillna(0)
        _standardize = lambda X: (X - X.wmean(Wa)) / np.sqrt(X.wvar(Wa))
        Y = dh.y.pipe(makesw)
        Ystd = Y.pipe(_standardize)
        Ya = Y.fillna(0)
        Ystda = Ystd.fillna(0)
        _demean = lambda x: x - x.wmean(Wa)
    dict.update(namespace, **var_map)
    

In [ ]:
def def_WY(dh):
    pass